In [ ]:
import sys, logging
log_format = '%(asctime)s | %(levelname)s : %(message)s'
logging.basicConfig(format=log_format, level=logging.INFO, stream=sys.stdout)

import tensorflow as tf
logging.info(f"Start with tf version {tf.version.VERSION}")

from pathlib import Path
cxr_image_path = Path(os.environ['DATA_NIH_CXR8']) / 'images' / '00010827_000.png'
logging.info(f"Test radiograph at {cxr_image_path}")

In [ ]:
%matplotlib inline

import cv2
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(4,4))

from imageio import imread
from skimage.color import rgb2gray
from common.image_ops import *

sz = 128
cxr_image = imread(cxr_image_path)
cxr_image = resize(cxr_image, (sz,sz))
cxr_image = img_as_uint(cxr_image)
logging.info(np.histogram(cxr_image, 10))

cxr_image = clahe.apply(cxr_image)
cx_min, cx_max = np.min(cxr_image), np.max(cxr_image)
cx_width = cx_max - cx_min
cxr_image = (cxr_image - cx_min) / cx_width
cxr_image = np.reshape(cxr_image, (1,sz,sz,1))
logging.info(np.histogram(cxr_image, 10))

In [ ]:
from common.filter_banks import *
ks = wave_numbers(6)
gabor_kernels = \
    make_gabor_kernels(sz=13, ks=ks, freqs=[0.5,1.0,2.0])
logging.info(f"Generated gabor kernels of size={gabor_kernels.shape}")

In [ ]:
cxr_image_tf = tf.constant(cxr_image, dtype=tf.float32)
cxr_image_out_tf_sq = conv2d_sq(cxr_image_tf, gabor_kernels)
cxr_image_out_sub = \
    tf.nn.max_pool(cxr_image_out_tf_sq, (4,4), 
                   strides=2, padding='SAME')

show_filter_response(cxr_image_out_sub, rows=3)

In [ ]:
current_value = tf.Variable(0.5 * np.ones((1,sz,sz,1)), dtype=tf.float32)
with tf.GradientTape() as tape:
    current_out = mse_tf(current_value, gabor_kernels, cxr_image_out_sub)
grad = tape.gradient(current_out, current_value)
print(grad.shape, current_out)
fig, axs = plt.subplots(1,2,figsize=(5,3))
axs[0].imshow(tf.reshape(current_value, (sz,sz)))
axs[1].imshow(tf.reshape(grad, (sz,sz)))

In [ ]:
%matplotlib inline
from IPython.display import clear_output
learning_rate = 1e+3

optimizer = tf.optimizers.Adam()
for n in range(1000):
    optimizer.apply_gradients(zip([grad], [current_value]))
    # current_value.assign_sub(learning_rate * grad)
    with tf.GradientTape() as tape:
        current_out = mse_tf(current_value, gabor_kernels, cxr_image_out_sub)
        # tf.print(f"loss = {current_out}   ")
    grad = tape.gradient(current_out, current_value)
    
fig, axs = plt.subplots(1,3,figsize=(5,3))
axs[0].imshow(tf.reshape(cxr_image[...,0], (sz,sz)), cmap='gray', vmin=0., vmax=0.8)
axs[1].imshow(tf.reshape(current_value, (sz,sz)), cmap='gray', vmin=0., vmax=0.8)
axs[2].imshow(tf.reshape(grad, (sz,sz)))

## Now do the same thing for a CT slice

In [ ]:
%matplotlib inline
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(4,4))
slice_path = Path('g:/') / 'Data_Training_MedicalImaging' / 'NIH_DeepLesion' \
                    / 'Images_png_01' / 'Images_png' \
                    / '000001_01_01' / '108.png'
transverse_slice = imread(slice_path)
transverse_slice = resize(transverse_slice, (sz,sz))
transverse_slice = img_as_uint(transverse_slice)
logging.info(np.histogram(transverse_slice, 10))

transverse_slice = clahe.apply(transverse_slice)
ts_min, ts_max = np.min(transverse_slice), np.max(transverse_slice)
ts_width = ts_max - ts_min
transverse_slice = (transverse_slice - ts_min) / ts_width
transverse_slice = np.reshape(transverse_slice, (1,sz,sz,1))
logging.info(np.histogram(transverse_slice, 10))

transverse_slice_tf = tf.constant(transverse_slice, dtype=tf.float32)
transverse_slice_out_tf_sq = conv2d_sq(transverse_slice_tf, gabor_kernels)
transverse_slice_out_sub = \
    tf.nn.max_pool(transverse_slice_out_tf_sq, (2,2), 
                   strides=2, padding='SAME')

show_filter_response(transverse_slice_out_sub, rows=3)